# Neural Network

In [1]:
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

import yaml

import matplotlib.pyplot as plt

In [2]:
hparams = yaml.safe_load(open('../src/models/hparams.yaml'))

hparams

{'learning_rate': 0.01, 'batch_size': 16, 'num_hidden_layers': 4}

In [3]:
# todo: change this to binary format.

wiki_df = pd.read_csv('../data/processed/wiki_df.csv', sep=';')

wiki_df.head()

,full_path,gender,age,img_array
0,17/10000217_1981-05-05_2009.jpg,1.0,28,[255 255 255 ... 144 78 27]
1,12/100012_1948-07-03_2008.jpg,1.0,60,[92 98 93 ... 35 31 30]
2,16/10002116_1971-05-31_2012.jpg,0.0,41,[ 10 30 61 ... 231 237 255]
3,02/10002702_1960-11-09_2012.jpg,0.0,52,[178 122 97 ... 168 112 83]
4,41/10003541_1937-09-27_1971.jpg,1.0,34,[194 189 190 ... 101 103 104]


In [4]:
import cv2

# Noramalize image values on range <0.0;1.0>
# check: https://www.tensorflow.org/tutorials/images/cnn
#wiki_df['img_array'] = wiki_df['img_array'] / 255.0
#wiki_df['img_array'] = wiki_df['full_path'].apply(lambda x: np.array(cv2.resize(cv2.imread('../data/raw/wiki_crop/' + x), (224, 224), interpolation=cv2.INTER_LINEAR).reshape(1, -1)[0]))

# TODO CHECK:
# wiki_df['img_array'] = wiki_df['full_path'].apply(lambda x: cv2.resize(cv2.imread('../data/raw/wiki_crop/' + x), (224, 224), interpolation=cv2.INTER_LINEAR).reshape(1, -1)[0])

wiki_df = wiki_df.drop(['img_array'], axis=1)

## Experiment 1

* Ako prvé sa pokúsime vytvoriť NN podobnú VGG. Podľa [WEEK_7 lab](https://github.com/matus-pikuliak/neural_networks_at_fiit/blob/92b24eef8e6444c43a22e8fa51a349b3b1043a7c/week_7/week_7.ipynb), alebo iného tutoriálu
* Natrénujeme ju na už predspracovanom datasete
* Jej výsledok pou%zijeme ako štartovaciu čiaru
* Túto sieť budeme rozširovať o ďalšie vrstvy a parametre
* V projekte ponecháme sieť s najlepším skóre
* Dole pripájam referenčnú ukážku siete. [Zdroj](https://www.pyimagesearch.com/2019/10/28/3-ways-to-create-a-keras-model-with-tensorflow-2-0-sequential-functional-and-model-subclassing/?__s)

In [5]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout


class MiniVGGNetModel(keras.Model):
    def __init__(self, classes, chanDim=-1):
        # call the parent constructor
        super(MiniVGGNetModel, self).__init__()

        # initialize the layers in the first (CONV => RELU) * 2 => POOL
        # layer set
        self.conv1A = Conv2D(32, (3, 3), padding="same")
        self.act1A = Activation("relu")
        self.bn1A = BatchNormalization(axis=chanDim)
        self.conv1B = Conv2D(32, (3, 3), padding="same")
        self.act1B = Activation("relu")
        self.bn1B = BatchNormalization(axis=chanDim)
        self.pool1 = MaxPooling2D(pool_size=(2, 2))

        # initialize the layers in the second (CONV => RELU) * 2 => POOL
        # layer set
        self.conv2A = Conv2D(32, (3, 3), padding="same")
        self.act2A = Activation("relu")
        self.bn2A = BatchNormalization(axis=chanDim)
        self.conv2B = Conv2D(32, (3, 3), padding="same")
        self.act2B = Activation("relu")
        self.bn2B = BatchNormalization(axis=chanDim)
        self.pool2 = MaxPooling2D(pool_size=(2, 2))

        # initialize the layers in our fully-connected layer set
        self.flatten = Flatten()
        self.dense3 = Dense(512)
        self.act3 = Activation("relu")
        self.bn3 = BatchNormalization()
        self.do3 = Dropout(0.5)

        # initialize the layers in the softmax classifier layer set
        self.dense4 = Dense(classes)
        self.softmax = Activation("softmax")

    def call(self, inputs):
        # build the first (CONV => RELU) * 2 => POOL layer set
        x = self.conv1A(inputs)
        x = self.act1A(x)
        x = self.bn1A(x)
        x = self.conv1B(x)
        x = self.act1B(x)
        x = self.bn1B(x)
        x = self.pool1(x)

        # build the second (CONV => RELU) * 2 => POOL layer set
        x = self.conv2A(x)
        x = self.act2A(x)
        x = self.bn2A(x)
        x = self.conv2B(x)
        x = self.act2B(x)
        x = self.bn2B(x)
        x = self.pool2(x)

        # build our FC layer set
        x = self.flatten(x)
        x = self.dense3(x)
        x = self.act3(x)
        x = self.bn3(x)
        x = self.do3(x)

        # build the softmax classifier
        x = self.dense4(x)
        x = self.softmax(x)

        # return the constructed model
        return x

In [6]:
#Generate target classes

classes = 101 #0 to 100
target = wiki_df['age'].values           ### train/valid Y
target_classes = keras.utils.to_categorical(target, classes)

file_paths = wiki_df['full_path'].values ### train/valid X

N_SAMPLES = len(target)

assert(len(target) == len(file_paths))

len(target_classes)

22578

In [7]:
# Load single image from disk to memmory
# resize and convert to np array
def load_img(x):
    im = cv2.imread('../data/raw/wiki_crop/' + x)
    im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    #im = im.reshape(1, -1)[0]
    return (np.array(im) / 255.0).astype(np.float32)


In [8]:
model = MiniVGGNetModel(
    classes = classes)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

callbacks = [
    #keras.callbacks.TensorBoard(
    #    log_dir=os.path.join("logs", timestamp()),
    #    histogram_freq=1,
    #    profile_batch=0)
]

# callbacks = []  # If you do not want to log results into TensorBoard

scores = []
epochs = 60
batch_size = 64

In [12]:
print("samples", N_SAMPLES)

train = 3000
validation = 1000

def load_image_data(files):
    data = []
    for file in files:
        data.append(load_img(file))
    return np.array(data)

def load_data(range=(0,0)):
    x = file_paths[range[0]:range[1]]
    x = load_image_data(x)

    # https://stackoverflow.com/questions/49083984/valueerror-can-not-squeeze-dim1-expected-a-dimension-of-1-got-3-for-sparse
    # sparse_categorical_crossentropy
    y = target[range[0]:range[1]] # target_classes
    
    return (x, y)
    
train_images, train_labels = load_data((0, train))
test_images, test_labels = load_data((train, train+validation))

samples 22578


In [10]:
# from: https://stackoverflow.com/a/55666861
def change_to_right(wrong_labels):
    right_labels=[]
    for x in wrong_labels:
        for i in range(0,len(wrong_labels[0])):
            if x[i]==1:
                right_labels.append(i)
    return right_labels

#train_labels = tf.convert_to_tensor(np.array(change_to_right(train_labels)))
#test_labels = tf.convert_to_tensor(np.array(change_to_right(test_labels)))

In [13]:
### train

score = model.fit(
    x=train_images,
    y=train_labels,
    batch_size = batch_size,
    validation_data = (test_images, test_labels),
    callbacks = callbacks,
    epochs = epochs)

scores.append(score)

model.summary()  # Writes number of parameters for each layer at the end of the training

Train on 3000 samples, validate on 1000 samples
Epoch 1/60
3000/3000 [==============================] - 256s 85ms/sample - loss: 5.1940 - accuracy: 0.1047 - val_loss: 5.5953 - val_accuracy: 0.0060
Epoch 2/60
3000/3000 [==============================] - 255s 85ms/sample - loss: 3.1931 - accuracy: 0.2913 - val_loss: 5.9673 - val_accuracy: 0.0010
Epoch 3/60
3000/3000 [==============================] - 263s 88ms/sample - loss: 1.7598 - accuracy: 0.6010 - val_loss: 6.2476 - val_accuracy: 0.0090
Epoch 4/60
3000/3000 [==============================] - 263s 88ms/sample - loss: 0.6561 - accuracy: 0.8723 - val_loss: 6.6410 - val_accuracy: 0.0270
Epoch 5/60
3000/3000 [==============================] - 262s 87ms/sample - loss: 0.2063 - accuracy: 0.9773 - val_loss: 5.2258 - val_accuracy: 0.0280
Epoch 6/60
3000/3000 [==============================] - 251s 84ms/sample - loss: 0.0747 - accuracy: 0.9960 - val_loss: 4.6817 - val_accuracy: 0.0300
Epoch 7/60
3000/3000 [==============================] - 24

KeyboardInterrupt: 

In [14]:
#Save model

tf.keras.models.save_model(
    model,
    filepath = "../models/mini-vgg-1.tf",
    overwrite=True,
    include_optimizer=True,
    save_format="tf"
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../models/mini-vgg-1.tf/assets


# Experiment 2
Cely VGG model.

Zdroj: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

In [5]:
model = keras.models.Sequential()
model.add(keras.layers.ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(keras.layers.Convolution2D(64, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(128, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.Convolution2D(4096, (7, 7), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Convolution2D(4096, (1, 1), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Convolution2D(2622, (1, 1)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Activation('softmax'))

In [7]:
model.load_weights('../data/vgg_face_weights.h5')

In [45]:
vgg_face_descriptor = keras.Model(inputs=model.layers[0].input, outputs=model.layers[-1].output)

Convert image as array using opencv

In [59]:
classes = 101 #0 to 100
target = wiki_df['age'].values
target_classes = keras.utils.to_categorical(target, classes)

features = []
 
for i in range(0, 1):
    features.append(wiki_df['img_array'].values[i])
 
features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)

In [54]:
for layer in model.layers[:-7]:
    layer.trainable = False

In [56]:
base_model_output = keras.models.Sequential()
base_model_output = keras.layers.Convolution2D(101, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = keras.layers.Flatten()(base_model_output)
base_model_output = keras.layers.Activation('softmax')(base_model_output)

In [58]:
age_model = keras.Model(inputs=model.input, outputs=base_model_output)

In [63]:
age_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
 
scores = []
epochs = 250; batch_size = 256
 
for i in range(epochs):
    print("epoch ",i)
 
ix_train = np.random.choice(train_x.shape[0], size=batch_size)
 
score = age_model.fit(train_x[ix_train], train_y[ix_train], epochs=1, validation_data=(test_x, test_y))
 
scores.append(score)

epoch  0
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
epoch  11
epoch  12
epoch  13
epoch  14
epoch  15
epoch  16
epoch  17
epoch  18
epoch  19
epoch  20
epoch  21
epoch  22
epoch  23
epoch  24
epoch  25
epoch  26
epoch  27
epoch  28
epoch  29
epoch  30
epoch  31
epoch  32
epoch  33
epoch  34
epoch  35
epoch  36
epoch  37
epoch  38
epoch  39
epoch  40
epoch  41
epoch  42
epoch  43
epoch  44
epoch  45
epoch  46
epoch  47
epoch  48
epoch  49
epoch  50
epoch  51
epoch  52
epoch  53
epoch  54
epoch  55
epoch  56
epoch  57
epoch  58
epoch  59
epoch  60
epoch  61
epoch  62
epoch  63
epoch  64
epoch  65
epoch  66
epoch  67
epoch  68
epoch  69
epoch  70
epoch  71
epoch  72
epoch  73
epoch  74
epoch  75
epoch  76
epoch  77
epoch  78
epoch  79
epoch  80
epoch  81
epoch  82
epoch  83
epoch  84
epoch  85
epoch  86
epoch  87
epoch  88
epoch  89
epoch  90
epoch  91
epoch  92
epoch  93
epoch  94
epoch  95
epoch  96
epoch  97
epoch  98
epoch  99
epoch  100

KeyboardInterrupt: 